In [1]:
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import viridis

/Users/rutujajangle/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
df = pd.read_csv('Combined.csv')
duration_mins_secs = [str(f"{i}:{j}") for i, j in zip([int(i//1) for i in list(df['Duration'])], [int(round((i%1) * 60, 2)) for i in list(df['Duration'])])]
df['Track Duration'] = duration_mins_secs
df.sample(5)

,Unnamed: 0,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,...,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Playlist,Track Duration
38,38,Northern Downpour,spotify:track:0tutU5E5kMR3bY8sZllpoJ,https://open.spotify.com/track/0tutU5E5kMR3bY8...,['Panic! At The Disco'],Pretty. Odd.,47,4.13,170.173,5,...,0.474,-9.406,0.0262,0.06430,0.000000,0.0645,0.241,4,Playlist 1,4:7
47,47,Cough Syrup,spotify:track:1UqhkbzB1kuFwt2iy4h29Q,https://open.spotify.com/track/1UqhkbzB1kuFwt2...,['Young the Giant'],Young The Giant (Special Edition),67,4.16,128.978,11,...,0.721,-7.307,0.0417,0.03430,0.000006,0.1150,0.225,3,Playlist 1,4:9
26,26,Use Somebody,spotify:track:5VGlqQANWDKJFl0MBG3sg2,https://open.spotify.com/track/5VGlqQANWDKJFl0...,['Kings of Leon'],Only By The Night,76,3.85,137.028,0,...,0.715,-5.356,0.0432,0.00552,0.000417,0.2010,0.173,4,Playlist 1,3:51
36,36,Yesterday - Remastered 2009,spotify:track:3BQHpFgAp4l80e1XslIjNI,https://open.spotify.com/track/3BQHpFgAp4l80e1...,['The Beatles'],Help! (Remastered),76,2.09,96.524,5,...,0.179,-11.830,0.0326,0.87900,0.000000,0.0886,0.315,4,Playlist 1,2:5
45,45,Friday I'm In Love,spotify:track:4QlzkaRHtU8gAdwqjWmO8n,https://open.spotify.com/track/4QlzkaRHtU8gAdw...,['The Cure'],Wish,75,3.59,136.205,0,...,0.763,-12.663,0.0341,0.00126,0.000068,0.3590,0.514,4,Playlist 1,3:35


In [3]:
p1 = pd.read_csv('Playlist1.csv')
p1 = pd.DataFrame(p1[['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Speechiness', 'Valence']].mean()).T
p2 = pd.read_csv('Playlist2.csv')
p2 = pd.DataFrame(p2[['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Speechiness', 'Valence']].mean()).T
p = pd.concat([p1, p2], ignore_index=True)
p

,Acousticness,Danceability,Energy,Instrumentalness,Speechiness,Valence
0,0.323185,0.49870,0.54508,0.051621,0.035824,0.344830
1,0.134489,0.64832,0.71414,0.012601,0.061262,0.556962


In [4]:
artists = pd.read_csv('All Artists.csv')
artists['Index'] = artists.index
artists.sample(5)

,Name,ID,Genres,Followers,Popularity,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2,Index
55,Miley Cyrus,5YGY8feqx7naU7z4HrwZM6,['pop'],25056264,84,Playlist 2,1,0,1,55
78,Bruno Mars,0du5cEVh5yTK9QJze8zA0C,"['dance pop', 'pop']",62300171,96,Playlist 2,2,0,2,78
50,Bastille,7EQ0qTo7fWT7DPxmxtSYEc,"['metropopolis', 'modern rock', 'pop']",5585188,76,Playlist 2,2,0,2,50
82,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,"['art pop', 'dance pop', 'pop']",32798999,92,Playlist 2,1,0,1,82
24,The Beatles,3WrFJ7ztbogyGnTHbHJFl2,"['british invasion', 'classic rock', 'merseybe...",28780119,86,Playlist 1,3,3,0,24


In [5]:
genres = pd.read_csv('Genres.csv')
genres['Index'] = genres.index
genres.sample(5)

,Genre,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2,Index
13,etherpop,Playlist 2,1,0,1,13
67,punk,Playlist 1,1,1,0,67
49,uk dance,Playlist 2,1,0,1,49
54,lilith,Playlist 1,1,1,0,54
24,british singer-songwriter,Playlist 1,1,1,0,24


In [6]:
collabs = pd.read_csv('All Collaborations.csv')
collabs.sample(5)

,Artist 1,Artist 2,Popular Collab,URL,Popularity,Artist From
102,Kygo,Selena Gomez,It Ain’t Me (with Selena Gomez),https://open.spotify.com/track/12GEpg2XOPyqk03...,64.0,Artists from Same Playlist
83,Selena Gomez,Tainy,"I Can’t Get Enough (benny blanco, Selena Gomez...",https://open.spotify.com/track/1WSGTYKLEpKfNQY...,58.0,Artists from Same Playlist
67,Clean Bandit,Taylor Swift,You Need To Calm Down - Clean Bandit Remix,https://open.spotify.com/track/7Bjf96uNizpvXLv...,44.0,Artists in Different Playlists
263,Zedd,Martin Garrix,Follow,https://open.spotify.com/track/5aXTfpNOqLj35yd...,52.0,Artists from Same Playlist
246,Ariana Grande,Lizzo,Good as Hell (feat. Ariana Grande),https://open.spotify.com/track/2rjjNUYONZqC7wS...,56.0,Artists from Same Playlist


In [7]:
artists_songs = pd.read_csv("Common Artist Songs.csv")
artists_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Artist
17,This Love,spotify:track:6ECp64rv50XVz93WvxXMGF,https://open.spotify.com/track/6ECp64rv50XVz93...,['Maroon 5'],Songs About Jane: 10th Anniversary Edition,82,3.44,95.051,5,0,0.712,0.862,-4.612,0.0378,0.052500,0.000000,0.093,0.809,4,Maroon 5
0,Yellow,spotify:track:3AJwUDP919kvQ9QcozQPxg,https://open.spotify.com/track/3AJwUDP919kvQ9Q...,['Coldplay'],Parachutes,89,4.45,173.372,11,1,0.429,0.661,-7.227,0.0281,0.002390,0.000121,0.234,0.285,4,Coldplay
8,Alone,spotify:track:1FlKUh04wsO7Z2AFmzplCw,https://open.spotify.com/track/1FlKUh04wsO7Z2A...,['The Cure'],Songs Of A Lost World,67,6.81,100.009,9,0,0.453,0.884,-5.051,0.0315,0.000022,0.325000,0.334,0.140,4,The Cure
32,I Can Do It With a Broken Heart,spotify:track:4q5YezDOIPcoLr8R81x9qy,https://open.spotify.com/track/4q5YezDOIPcoLr8...,['Taylor Swift'],THE TORTURED POETS DEPARTMENT,83,3.63,129.994,0,1,0.701,0.751,-8.390,0.0502,0.123000,0.000000,0.150,0.220,4,Taylor Swift
3,A Sky Full of Stars,spotify:track:0FDzzruyVECATHXKHFs9eJ,https://open.spotify.com/track/0FDzzruyVECATHX...,['Coldplay'],Ghost Stories,85,4.46,124.970,6,1,0.545,0.675,-6.474,0.0279,0.006170,0.001970,0.209,0.162,4,Coldplay


In [8]:
genre_songs = pd.read_csv("Common Genre Songs.csv")
genre_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Genre
278,No. 1 Party Anthem,spotify:track:5TTGoX70AFrTvuEtqHK37S,https://open.spotify.com/track/5TTGoX70AFrTvuE...,['Arctic Monkeys'],AM,82,4.05,115.013,0,1,0.505,0.698,-5.563,0.0297,0.3300,0.088700,0.0966,0.599,4,permanent wave
290,Heart-Shaped Box,spotify:track:11LmqTE2naFULdEP94AUBa,https://open.spotify.com/track/11LmqTE2naFULdE...,['Nirvana'],In Utero (Deluxe Edition),79,4.69,203.006,1,1,0.256,0.641,-10.283,0.0552,0.1990,0.032900,0.0853,0.382,4,permanent wave
77,One Step Closer,spotify:track:3K4HG9evC7dg3N0R9cYqk4,https://open.spotify.com/track/3K4HG9evC7dg3N0...,['Linkin Park'],Hybrid Theory (Bonus Edition),82,2.62,95.136,9,1,0.492,0.969,-4.419,0.0491,0.0014,0.000001,0.0787,0.538,4,rock
330,Take Me to Church,spotify:track:1CS7Sd1u5tWkstBhpssyjP,https://open.spotify.com/track/1CS7Sd1u5tWkstB...,['Hozier'],Hozier (Expanded Edition),80,4.03,128.945,4,0,0.566,0.664,-5.303,0.0464,0.6340,0.000000,0.1160,0.437,4,pov: indie
165,How to Save a Life,spotify:track:5fVZC9GiM4e8vu99W0Xf6J,https://open.spotify.com/track/5fVZC9GiM4e8vu9...,['The Fray'],How To Save A Life,80,4.38,122.035,10,1,0.640,0.743,-4.080,0.0379,0.2690,0.000000,0.1010,0.361,4,piano rock


In [9]:
new_songs = pd.read_csv("New Playlist.csv")
duration_mins_secs = [str(f"{i}:{j}") for i, j in zip([int(i//1) for i in list(new_songs['Duration'])], [int(round((i%1) * 60, 2)) for i in list(new_songs['Duration'])])]
new_songs['Track Duration'] = duration_mins_secs
new_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Track Duration
44,Basket Case,spotify:track:6L89mwZXSOwYl76YXfX13s,https://open.spotify.com/track/6L89mwZXSOwYl76...,['Green Day'],Dookie,80,3.03,85.064,3,1,0.442,0.943,-3.205,0.0602,0.00293,0.000009,0.0910,0.7810,4,3:1
274,this is what falling in love feels like,spotify:track:2PWTZV5znjLtZC5T1EVJvL,https://open.spotify.com/track/2PWTZV5znjLtZC5...,['JVKE'],this is what ____ feels like (Vol. 1-4),75,2.01,128.934,11,1,0.422,0.440,-5.530,0.0544,0.61700,0.000000,0.0837,0.3320,5,2:0
220,You & Me - Rivo Remix,spotify:track:0xoYZ45fgTfyQYREZPN7Sa,https://open.spotify.com/track/0xoYZ45fgTfyQYR...,"['Disclosure', 'Eliza Doolittle', 'Rivo']",You & Me (Rivo Remix),79,3.48,122.011,10,0,0.702,0.797,-6.635,0.0557,0.58300,0.495000,0.1790,0.0356,4,3:28
108,Joyride - Single Version,spotify:track:3Skwc4sh1QHUGcWrFBFWSF,https://open.spotify.com/track/3Skwc4sh1QHUGcW...,['Roxette'],Don't Bore Us - Get to the Chorus! Roxette's G...,56,4.03,102.233,2,1,0.651,0.740,-6.244,0.0272,0.08620,0.000000,0.1420,0.6150,4,4:1
208,Kiss Me,spotify:track:1ELMCVbADOtZj8tYdS01nI,https://open.spotify.com/track/1ELMCVbADOtZj8t...,['Dermot Kennedy'],Sonder (2023),67,3.82,120.082,0,0,0.645,0.843,-2.210,0.0347,0.00106,0.000000,0.0650,0.4660,4,3:49


In [10]:
cols = ['Popularity', 'Tempo', 'Danceability', 'Energy', 'Acousticness', 'Valence']
scatter_cols = ['Acousticness', 'Danceability', 'Energy', 'Tempo', 'Valence']
hist_cols = ['Popularity', 'Tempo', 'Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveliness', 'Valence']

---

In [11]:
app1 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app1.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='left', children=[
            dbc.Label("Comparing Playlists - CDF Plots"),
            dcc.Graph(id="histogram")
        ]),
        dbc.Col(width=2, align='right', children=[
            html.Div([
                html.Label('Field'),
                dcc.Dropdown(id='field', value=hist_cols[0], options=[{'label': col, 'value': col} for col in hist_cols])
            ]),
            html.Div([
                html.Label('Bins'),
                dcc.Slider(id='nbins', min=5, max=200, marks=None, step=1, value=10)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, marks=None, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app1.callback(
    Output("histogram", "figure"), 
    [Input("field", "value"),
     Input('nbins', 'value'), 
     Input('opacity', 'value')]
)

def update_chart(field, nbins, opacity):
    fig = px.histogram(df, x=field, histfunc='count', nbins=nbins, color='Playlist', color_discrete_sequence=['cyan', 'lightpink'], opacity=opacity, cumulative=True, width=1400, height=600)
    fig.update_layout(
        barmode='overlay',
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app1.run(port=8855, debug=True)

---

In [12]:
app2 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app2.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Scatterplot"),
            dcc.Graph(id="scatter")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('X-Axis'),
                dcc.Dropdown(id='xax', value=scatter_cols[0], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Y-Axis'),
                dcc.Dropdown(id='yax', value=scatter_cols[1], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Size'),
                dcc.Dropdown(id='size', value=scatter_cols[2], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app2.callback(
    Output("scatter", "figure"), 
    [Input("xax", "value"),
     Input('yax', 'value'),
     Input('size', 'value'),
     Input('opacity', 'value')]
)

def update_figure(xax, yax, size, opacity):
    hover_dict = {xax: False, yax:False, size:False, 'Playlist':False, 'Album':True, 'Artist(s)':True, 'Track Duration':True, 'Popularity':True}
    fig = px.scatter(df, x=xax, y=yax, size=size, color='Playlist', trendline='ols', color_discrete_sequence=['cyan', 'lightpink'], hover_name="Track_Name", hover_data=hover_dict, size_max=25, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity=opacity)
    return fig

app2.run(port=8856, debug=True)

---

In [13]:
app3 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app3.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Radar Plot"),
            dcc.Graph(id="radar"),
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5, marks=None, step=None)
            ]),
        ])
    ])
], fluid=True)

@app3.callback(
    Output("radar", "figure"),
    Input('opacity', 'value')
)

def update_figure(opacity):
    trace1 = go.Scatterpolar(
        r = list(p.iloc[0, :]),
        theta = list(p.columns),
        fill = 'toself',
        fillcolor = 'cyan',
        line = dict(color='black', width=1),
        name='Playlist 1'
    )
    trace2 = go.Scatterpolar(
        r = list(p.iloc[1, :]),
        theta = list(p.columns),
        fill = 'toself',
        fillcolor = 'lightpink',
        line = dict(color='black', width=1),
        name = 'Playlist 2'
    )
    layout = go.Layout(
        polar = dict(
            radialaxis = dict(
                visible = True,
                range = [0, 1]
            )
        ),
        showlegend = True,
        height = 600,
        width = 1400,
        plot_bgcolor = 'rgba(0, 0, 0, 0)'
    )
    fig = go.Figure(data=[trace1, trace2], layout=layout)
    fig.update_layout(
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity = opacity)
    return fig

app3.run(port=8857, debug=True)

---

In [14]:
app4 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app4.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Top Artists"),
            dcc.Graph(id="bubble")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value='Followers', options=[{'label': col, 'value': col} for col in ['Followers', 'Popularity']])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app4.callback(
    Output("bubble", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value'),
     Input('opacity', 'value')]
)

def update_figure(measure, topn, opacity):
    hover_dict = {measure: False, 'Index':False, 'Name':False, 'Playlist':False, 'Genres':True, 'Popularity':True, 'Followers':True, 'Total Count':False}
    fig = px.scatter(artists.sort_values(by=measure, ascending=False).head(topn), x='Index', y=measure, text='Name', size='Total Count', size_max=100, color='Playlist', color_discrete_map={'Playlist 1':'cyan', 'Playlist 2':'lightpink', 'Both':'purple'}, hover_name="Name", hover_data=hover_dict, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app4.run(port=8858, debug=True)

---

In [15]:
app5 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app5.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Top Genres"),
            dcc.Graph(id="bubble_too")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app5.callback(
    Output("bubble_too", "figure"),
     [Input('topn', 'value'),
     Input('opacity', 'value')]
)

def update_figure(topn, opacity):
    hover_dict = {'Index':False, 'Genre':False, 'Playlist':False, 'Total Count':True}
    fig = px.scatter(genres.sort_values(by='Total Count', ascending=False).head(topn), x='Total Count', y='Index', text='Genre', size='Total Count', size_max=100, color='Playlist', color_discrete_map={'Playlist 1':'cyan', 'Playlist 2':'lightpink', 'Both':'purple'}, hover_name="Genre", hover_data=hover_dict, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel=dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app5.run(port=8859, debug=True)

---

In [16]:
app6 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app6.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Collaboration Network"),
            dcc.Graph(id="network")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ])
        ])
    ])
], fluid=True)

@app6.callback(
    Output("network", "figure"),
     Input('opacity', 'value')
)

def update_figure(opacity):

    G = nx.Graph()
    for i, j, k, l, m in zip(list(artists['Name']), list(artists['Genres']), list(artists['Popularity']), list(artists['Followers']), list(artists['Playlist'])):
        G.add_node(i)
        G.nodes[i]['Genres'] = j
        G.nodes[i]['Popularity'] = k
        G.nodes[i]['Followers'] = l
        G.nodes[i]['Playlist'] = m
    for i, j, k, l, m in zip(list(collabs['Artist 1']), list(collabs['Artist 2']), list(collabs['Popular Collab']), list(collabs['Popularity']), list(collabs['Artist From'])):
        G.add_edge(i, j)
        G.edges[(i, j)]['Popular Collab'] = k
        G.edges[(i, j)]['Popularity'] = l
        G.edges[(i, j)]['Artist From'] = m
    nx.write_gexf(G, 'Collaboration Graph.gexf')

    pos = nx.spring_layout(G, k=0.99)
    pos = nx.rescale_layout_dict(pos, 1000)
    for n in G.nodes:
        G.nodes[n]['pos'] = pos[n]

    edge_x = []
    edge_y = []
    xtext = []
    ytext = []
    edgetext = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        xtext.append((x0 + x1)/2)
        ytext.append((y0 + y1)/2)
        edgetext.append(f"Artist 1: {edge[0]}, \nArtist 2: {edge[1]},\n Collaboration: {G.edges[edge]['Popular Collab']}")
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, 
        y=edge_y,
        line = dict(
            width = 1, 
            color = '#888'
        ),
        hoverinfo='text',
        mode='lines'
    )
    edge_trace.text = edgetext

    eweights_trace = go.Scatter(
        x = xtext,
        y = ytext, 
        mode = 'markers',
        marker = dict(
            size = 3,
            color = '#888',
        ),
        text = edgetext,
        hoverinfo = 'text'
    )

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
    
    node_trace = go.Scatter(
        x = node_x, 
        y = node_y,
        mode = 'markers',
        hoverinfo = 'text',
        marker = dict(
            color = [],
            size = [],
            line_width = 0.5
        )
    )

    pd = {'Playlist 1':'cyan', 'Playlist 2':'lightpink', 'Both':'purple'}

    node_size = []
    node_color = []
    node_text = []
    for n in G.nodes:
        node_size.append((nx.degree_centrality(G)[n] * 500) + 10)
        node_color.append(pd[G.nodes[n]['Playlist']])
        node_text.append(f"Name: {n},\nGenres: {G.nodes[n]['Genres']}")

    node_trace.marker.size = node_size
    node_trace.marker.color = node_color
    node_trace.text = node_text

    layout = go.Layout(
        showlegend = False,
        hovermode = 'closest',
        xaxis = dict(
            showgrid = False, 
            zeroline = False, 
            showticklabels = False
        ),
        yaxis = dict(
            showgrid = False, 
            zeroline = False, 
            showticklabels = False
        ),
        height=1100,
        width=1400,
        plot_bgcolor='rgba(0, 0, 0, 0)'
    )

    fig = go.Figure(data=[edge_trace, eweights_trace, node_trace], layout=layout)
    fig.update_layout(
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity = opacity)
    return fig

app6.run(port=8860, debug=True)

---

In [17]:
app7 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app7.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Wall of Recommendations - Common Genres"),
            dcc.Graph(id="treemap")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
        ])
    ])
], fluid=True)

@app7.callback(
    Output("treemap", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value')]
)

def update_figure(measure, topn):
    hover_dict = {'Artist(s)':True, measure:False, 'Track_Name':False}
    fig = px.treemap(
        genre_songs.sort_values(by=measure, ascending=False).head(topn), 
        path = ['Common Genre', 'Track_Name'],
        values = measure,
        color = measure,
        color_continuous_scale=px.colors.sequential.Reds,
        hover_name = 'Track_Name',
        hover_data = hover_dict,
    )
    fig.update_layout(
        width=1400,
        height=600,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        ),
    )
    return fig

app7.run(port=8861, debug=True)

---

In [18]:
app8 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app8.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Wall of Recommendations - Common Artists"),
            dcc.Graph(id="treemap_too")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
        ])
    ])
], fluid=True)

@app8.callback(
    Output("treemap_too", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value')]
)

def update_figure(measure, topn):
    hover_dict = {'Artist(s)':True, measure:False, 'Track_Name':False}
    fig = px.treemap(
        artists_songs.sort_values(by=measure, ascending=False).head(topn), 
        path = ['Common Artist', 'Track_Name'],
        values = measure,
        color = measure,
        color_continuous_scale=px.colors.sequential.Reds,
        hover_name = 'Track_Name',
        hover_data = hover_dict
    )
    fig.update_layout(
        width=1400,
        height=600,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app8.run(port=8862, debug=True)

---

In [19]:
app9 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app9.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Scatterplot"),
            dcc.Graph(id="scatter")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('X-Axis'),
                dcc.Dropdown(id='xax', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Y-Axis'),
                dcc.Dropdown(id='yax', value=cols[1], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Size'),
                dcc.Dropdown(id='size', value=cols[2], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Color'),
                dcc.Dropdown(id='color', value=cols[3], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5, step=None, marks=None)
            ]),
        ])
    ])
], fluid=True)

@app9.callback(
    Output("scatter", "figure"), 
    [Input("xax", "value"),
     Input('yax', 'value'),
     Input('size', 'value'),
     Input('color', 'value'),
     Input('opacity', 'value')]
)

def update_figure(xax, yax, size, color, opacity):
    hover_dict = {xax: False, yax:False, size:False, color:False, 'Album':True, 'Artist(s)':True, 'Track Duration':True, 'Popularity':True}
    fig = px.scatter(new_songs, x=xax, y=yax, size=size, color=color, trendline='ols', color_continuous_scale='Reds', hover_name="Track_Name", hover_data=hover_dict, size_max=15, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity=opacity)
    return fig

app9.run(port=8863, debug=True)

---

In [20]:
app10 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app10.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='left', children=[
            dbc.Label("New Playlist - CDF Plot"),
            dcc.Graph(id="histogram_too")
        ]),
        dbc.Col(width=2, align='right', children=[
            html.Div([
                html.Label('Field'),
                dcc.Dropdown(id='field', value=hist_cols[0], options=[{'label': col, 'value': col} for col in hist_cols])
            ]),
            html.Div([
                html.Label('Bins'),
                dcc.Slider(id='nbins', min=5, max=200, marks=None, step=1, value=10)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, marks=None, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app10.callback(
    Output("histogram_too", "figure"), 
    [Input("field", "value"),
     Input('nbins', 'value'), 
     Input('opacity', 'value')]
)

def update_chart(field, nbins, opacity):
    fig = px.histogram(new_songs, x=field, histfunc='count', nbins=nbins, opacity=opacity, cumulative=True, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app10.run(port=8864, debug=True)

---